In [107]:
import pandas as pd, sqlite3

conn = sqlite3.connect("../data/checking-logs.sqlite")

In [108]:
tables = pd.io.sql.read_sql("""
    SELECT name
    FROM sqlite_master
    WHERE type = 'table';
    """, conn)
print(tables)

        name
0  pageviews
1    checker
2  deadlines
3   datamart
4       test
5    control


In [109]:
## Results for test: before and after first_view_ts
query_test = """
SELECT
    CASE
        WHEN c.first_commit_ts < c.first_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(c.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS avg_diff
FROM test c
JOIN deadlines d ON c.labname = d.labs
WHERE c.labname != 'project1'
    AND EXISTS (
        SELECT 1 FROM test c2
        WHERE c2.uid = c.uid
            AND c2.first_commit_ts < c2.first_view_ts
    )
    AND EXISTS (
        SELECT 1 FROM test c2
        WHERE c2.uid = c.uid
            AND c2.first_commit_ts < c2.first_view_ts
    )
GROUP BY time;
"""
test_results = pd.read_sql(query_test, conn)

In [110]:
query_control = f"""
SELECT
    CASE
        WHEN c.first_commit_ts < avg_view_ts THEN 'before'
        ELSE 'after'
    END AS time,
    AVG((julianday(c.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS avg_diff
    SELECT *, {avg_ts_value} AS avg_view_ts FROM control
) c
JOIN deadlines d ON c.labname = d.labs
WHERE c.labname != 'project1'
  AND EXISTS (
    SELECT 1 FROM control c2
    WHERE c2.uid = c.uid
      AND c2.first_commit_ts < {avg_ts_value}
  )
  AND EXISTS (
    SELECT 1 FROM control c2
    WHERE c2.uid = c.uid
      AND c2.first_commit_ts > {avg_ts_value}
  )
GROUP BY time;
"""
control_results = pd.io.sql.read_sql(control_query, conn)
control_results.columns = ['time','AVG(diff)']

In [111]:
control_results

,time,AVG(diff)
0,after,-118.144425
1,before,-99.901295


In [112]:
test_results

,time,avg_diff
0,after,-94.473704
1,before,-61.156438


In [113]:
conn.close()